In [ ]:
import torch
import torch.nn as nn
from transformers import BertModel

class MultiTaskBERT(nn.Module):
    def __init__(self, num_sentiments, num_emotions, num_problems):
        super().__init__()

        self.bert = BertModel.from_pretrained("bert-base-uncased")

        hidden = self.bert.config.hidden_size

        self.sentiment_head = nn.Linear(hidden, num_sentiments)
        self.emotion_head = nn.Linear(hidden, num_emotions)
        self.problem_head = nn.Linear(hidden, num_problems)
        self.sarcasm_head = nn.Linear(hidden, 2)

    def forward(self, input_ids, attention_mask):
        output = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        pooled = output.pooler_output

        return {
            "sentiment": self.sentiment_head(pooled),
            "emotion": self.emotion_head(pooled),
            "problem": self.problem_head(pooled),
            "sarcasm": self.sarcasm_head(pooled)
        }


In [ ]:
import torch
import torch.nn as nn

loss_fn = nn.CrossEntropyLoss()

# These variables would typically be generated by a forward pass through your model
# and by loading corresponding ground truth labels from your dataset.

# Get number of unique classes for each task from the dataframe
# (Assuming df is available and populated from previous cells)
if 'df' in globals(): # Check if df is defined
    num_sentiments = df['sentiment'].nunique()
    num_emotions = df['emotion'].nunique()
    num_problems = df['type_of_problem'].nunique()
else:
    # Fallback if df is not globally available for some reason
    # (This branch should ideally not be hit given the notebook state)
    print("Warning: 'df' not found, using dummy class counts.")
    num_sentiments = 3 # Example dummy
    num_emotions = 6   # Example dummy
    num_problems = 4   # Example dummy

num_sarcasm = 2 # As defined in the MultiTaskBERT model's sarcasm_head

# Example: Define a batch size (this would be dynamic in a real training loop)
batch_size = 32

# Create dummy logits (model predictions) - replace with actual model output during training
sentiment_logits = torch.randn(batch_size, num_sentiments)
emotion_logits = torch.randn(batch_size, num_emotions)
problem_logits = torch.randn(batch_size, num_problems)
sarcasm_logits = torch.randn(batch_size, num_sarcasm)

# Create dummy labels (ground truth) - replace with actual data labels during training
sentiment_labels = torch.randint(0, num_sentiments, (batch_size,))
emotion_labels = torch.randint(0, num_emotions, (batch_size,))
problem_labels = torch.randint(0, num_problems, (batch_size,))
sarcasm_labels = torch.randint(0, num_sarcasm, (batch_size,))

loss = (
    loss_fn(sentiment_logits, sentiment_labels) +
    loss_fn(emotion_logits, emotion_labels) +
    loss_fn(problem_logits, problem_labels) +
    loss_fn(sarcasm_logits, sarcasm_labels)
)

print(f"Total loss calculated: {loss.item()}")

Total loss calculated: 7.41469669342041


In [ ]:
import pandas as pd

df = pd.read_csv("reviews.csv")

print("Sentiment classes:", df["sentiment"].unique())
print("Emotion classes:", df["emotion"].unique())
print("Problem classes:", df["type_of_problem"].unique())

print("\nCounts:")
print(df["sentiment"].value_counts())
print(df["emotion"].value_counts())
print(df["type_of_problem"].value_counts())


Sentiment classes: ['Negative' 'Positive' 'Neutral']
Emotion classes: ['Anger' 'Joy' 'Neutral' 'Surprise' 'Sadness' 'Disgust' 'Fear']
Problem classes: ['Shipping Delay' 'No Problem' 'Product Quality' 'Size/Fit Issue'
 'Wrong Item' 'Misleading Description' 'Poor Customer Service'
 'Defective Item' 'Damaged Product' 'Missing Parts']

Counts:
sentiment
Negative    2670
Positive    2668
Neutral     2662
Name: count, dtype: int64
emotion
Neutral     2595
Joy         1565
Surprise    1170
Anger       1068
Disgust      565
Sadness      538
Fear         499
Name: count, dtype: int64
type_of_problem
No Problem                2823
Product Quality           1210
Size/Fit Issue            1210
Shipping Delay             998
Missing Parts              316
Poor Customer Service      305
Damaged Product            293
Defective Item             290
Misleading Description     282
Wrong Item                 273
Name: count, dtype: int64


In [ ]:
from sklearn.preprocessing import LabelEncoder

sentiment_encoder = LabelEncoder()
emotion_encoder = LabelEncoder()
problem_encoder = LabelEncoder()

df["sentiment_label"] = sentiment_encoder.fit_transform(df["sentiment"])
df["emotion_label"] = emotion_encoder.fit_transform(df["emotion"])
df["problem_label"] = problem_encoder.fit_transform(df["type_of_problem"])


In [ ]:
import pickle

pickle.dump(sentiment_encoder, open("sentiment_encoder.pkl", "wb"))
pickle.dump(emotion_encoder, open("emotion_encoder.pkl", "wb"))
pickle.dump(problem_encoder, open("problem_encoder.pkl", "wb"))


In [ ]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(
    df,
    test_size=0.2,
    stratify=df["sentiment_label"],
    random_state=42
)


In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize(batch):
    return tokenizer(
        batch["review"],
        padding="max_length",
        truncation=True,
        max_length=128
    )


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
import torch
from torch.utils.data import Dataset

class ReviewDataset(Dataset):
    def __init__(self, df, tokenizer):
        self.df = df.reset_index(drop=True)
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        text = self.df.loc[idx, "review"]

        enc = self.tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=128,
            return_tensors="pt"
        )

        return {
            "input_ids": enc["input_ids"].squeeze(),
            "attention_mask": enc["attention_mask"].squeeze(),
            "sentiment_label": torch.tensor(self.df.loc[idx, "sentiment_label"]),
            "emotion_label": torch.tensor(self.df.loc[idx, "emotion_label"]),
            "problem_label": torch.tensor(self.df.loc[idx, "problem_label"])
        }


In [ ]:
import torch.nn as nn
from transformers import BertModel

class MultiTaskBERT(nn.Module):
    def __init__(self, num_sentiments, num_emotions, num_problems):
        super().__init__()

        self.bert = BertModel.from_pretrained("bert-base-uncased")
        hidden = self.bert.config.hidden_size

        self.sentiment_head = nn.Linear(hidden, num_sentiments)
        self.emotion_head = nn.Linear(hidden, num_emotions)
        self.problem_head = nn.Linear(hidden, num_problems)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        pooled = outputs.pooler_output

        return {
            "sentiment": self.sentiment_head(pooled),
            "emotion": self.emotion_head(pooled),
            "problem": self.problem_head(pooled)
        }


In [ ]:
loss_fn = nn.CrossEntropyLoss()

loss = (
    loss_fn(sentiment_logits, sentiment_labels) +
    loss_fn(emotion_logits, emotion_labels) +
    loss_fn(problem_logits, problem_labels)
)


In [ ]:
!pip install transformers torch scikit-learn pandas tqdm

In [ ]:
import pandas as pd

df = pd.read_csv("/content/reviews.csv")

df = df.dropna(subset=["review", "sentiment", "emotion", "type_of_problem"])
df.head()


,product_category,rating,review,type_of_problem,emotion,sentiment
0,Electronics,1,Regret this purchase. Product is defective and...,Shipping Delay,Anger,Negative
1,Toys & Games,5,Amazing purchase! Would definitely recommend t...,No Problem,Joy,Positive
2,Toys & Games,5,This exceeded all my expectations. Absolutely ...,No Problem,Neutral,Positive
3,Electronics,3,Very frustrated. The item arrived late and was...,Product Quality,Anger,Negative
4,Automotive,1,Poor quality materials. I expected much better...,Product Quality,Anger,Negative


In [ ]:
from sklearn.preprocessing import LabelEncoder
import pickle

sentiment_encoder = LabelEncoder()
emotion_encoder = LabelEncoder()
problem_encoder = LabelEncoder()

df["sentiment_label"] = sentiment_encoder.fit_transform(df["sentiment"])
df["emotion_label"] = emotion_encoder.fit_transform(df["emotion"])
df["problem_label"] = problem_encoder.fit_transform(df["type_of_problem"])

pickle.dump(sentiment_encoder, open("sentiment_encoder.pkl", "wb"))
pickle.dump(emotion_encoder, open("emotion_encoder.pkl", "wb"))
pickle.dump(problem_encoder, open("problem_encoder.pkl", "wb"))


In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")


In [ ]:
import torch
from torch.utils.data import Dataset

class ReviewDataset(Dataset):
    def __init__(self, df, tokenizer):
        self.df = df.reset_index(drop=True)
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        text = self.df.loc[idx, "review"]

        enc = self.tokenizer(
            text,
            padding="max_length",
            truncation=True,
            max_length=128,
            return_tensors="pt"
        )

        return {
            "input_ids": enc["input_ids"].squeeze(),
            "attention_mask": enc["attention_mask"].squeeze(),
            "sentiment_label": torch.tensor(self.df.loc[idx, "sentiment_label"]),
            "emotion_label": torch.tensor(self.df.loc[idx, "emotion_label"]),
            "problem_label": torch.tensor(self.df.loc[idx, "problem_label"])
        }


In [ ]:
import torch.nn as nn
from transformers import BertModel

class MultiTaskBERT(nn.Module):
    def __init__(self, n_sent, n_emotion, n_problem):
        super().__init__()
        self.bert = BertModel.from_pretrained("bert-base-uncased")
        hidden = self.bert.config.hidden_size

        self.sentiment = nn.Linear(hidden, n_sent)
        self.emotion = nn.Linear(hidden, n_emotion)
        self.problem = nn.Linear(hidden, n_problem)

    def forward(self, input_ids, attention_mask):
        out = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled = out.pooler_output

        return {
            "sentiment": self.sentiment(pooled),
            "emotion": self.emotion(pooled),
            "problem": self.problem(pooled)
        }


In [ ]:
from torch.utils.data import DataLoader
from tqdm import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"

model = MultiTaskBERT(
    len(sentiment_encoder.classes_),
    len(emotion_encoder.classes_),
    len(problem_encoder.classes_)
).to(device)

train_loader = DataLoader(ReviewDataset(train_df, tokenizer), batch_size=16, shuffle=True)
test_loader = DataLoader(ReviewDataset(test_df, tokenizer), batch_size=16)

optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
loss_fn = nn.CrossEntropyLoss()


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [21]:
for epoch in range(3):
    model.train()
    total_loss = 0

    for batch in tqdm(train_loader):
        optimizer.zero_grad()

        outputs = model(
            batch["input_ids"].to(device),
            batch["attention_mask"].to(device)
        )

        loss = (
            loss_fn(outputs["sentiment"], batch["sentiment_label"].to(device)) +
            loss_fn(outputs["emotion"], batch["emotion_label"].to(device)) +
            loss_fn(outputs["problem"], batch["problem_label"].to(device))
        )

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1} Loss: {total_loss/len(train_loader)}")


100%|██████████| 400/400 [2:45:25<00:00, 24.81s/it]


Epoch 1 Loss: 2.717011775970459


100%|██████████| 400/400 [2:44:15<00:00, 24.64s/it]


Epoch 2 Loss: 2.3097067523002623


100%|██████████| 400/400 [2:42:21<00:00, 24.35s/it]

Epoch 3 Loss: 2.2859276258945465
